<a href="https://colab.research.google.com/github/informatics-isi-edu/eye-ai-tools/blob/main/notebooks/AccessDiagnosis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Prerequisites
!pip install git+https://github.com/fair-research/bdbag
!pip install deriva
!pip install 'git+https://github.com/informatics-isi-edu/eye-ai-tools' --upgrade --force



  Cloning https://github.com/fair-research/bdbag to /tmp/pip-req-build-3f_76c9t
  Running command git clone --filter=blob:none --quiet https://github.com/fair-research/bdbag /tmp/pip-req-build-3f_76c9t
  Resolved https://github.com/fair-research/bdbag to commit 8ee285059caa71a0789179b8d57b03010ac00e7f
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/informatics-isi-edu/eye-ai-tools to /tmp/pip-req-build-0dbpltbp
  Running command git clone --filter=blob:none --quiet https://github.com/informatics-isi-edu/eye-ai-tools /tmp/pip-req-build-0dbpltbp
  Resolved https://github.com/informatics-isi-edu/eye-ai-tools to commit bda083a8fc3cbc965aafb3254da8d120a882dcac
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for ml-utils: filename=ml_utils-0.0.1-py3-none-any.whl size=9685 sha256=cae38eb5cc201baaa7f3e10d563fdba55b272b663fd1b13a9e1e094ee0550f46
  Stored in directory: /tm

In [20]:
# login to DERIVA via Globus Auth
DEBUG_MODE = False #@param
DEFAULT_SERVER = 'dev.eye-ai.org' if DEBUG_MODE else 'www.eye-ai.org'
!deriva-globus-auth-utils login --no-browser --host {DEFAULT_SERVER}

You are already logged in.


Make a connection to the Eye AI catalog and get a reference to the main schema: eye-ai

In [21]:
import pandas as pd
from ml_utils import EyeAI
from deriva.core import ErmrestCatalog, get_credential
protocol = 'https'
catalog_number = 'eye-ai'
credential = get_credential(DEFAULT_SERVER)
catalog = ErmrestCatalog(protocol, DEFAULT_SERVER, catalog_number, credential)
# Get the path builder interface for this catalog
pb = catalog.getPathBuilder()
eye_ai = pb.schemas['eye-ai']


EA = EyeAI(hostname = DEFAULT_SERVER, catalog_number = 'eye-ai')

In [22]:
def Reshape_table(frames, compare_value):

  # frames = [grader_diag, init_diag]
  long = pd.concat(frames).reset_index()
  # change data type for control vocab table
  cols = ['Image_Quality', 'Image_Side', 'Full_Name', 'Diagnosis']
  for c in cols:
    long[c] = long[c].astype('category')
  wide = pd.pivot(long, index=['Image', 'Image_Side','Subject_RID'], columns = 'Full_Name', values = compare_value) #Reshape from long to wide
  return long, wide

In [23]:
# @title Dataset Tags

pd.DataFrame(eye_ai.Dataset.path.attributes(eye_ai.Dataset.RID, eye_ai.Dataset.Description))

,RID,Description
0,1-EATE,Initial Rotterdam data set. Contains 101442 su...
1,2-0X9M,Rotterdam Train Subset for Cropping Algorithm ...
2,1-MGYM,Rotterdam dataset for evaluation. 20% of [1-EA...
3,1-NRJP,Rotterdam dataset for ML training. 64% of [1-E...
4,1-SQCE,Rotterdam dataset for ML testing. 16% of [1-EA...
5,2-1S12,**LAC Dataset** The initial data set from LAC....
6,2-1S14,**Invalid Dataset** The subjects in this datas...
7,2-277C,**Test Dataset** A subject dataset intended fo...
8,2-277E,**Development Dataset** All of the subjects in...
9,2-277G,**Training Dataset** A subject dataset that is...


In [24]:
# @title Diagnosis Tags
pd.DataFrame(eye_ai.Diagnosis_Tag.path.attributes(
    eye_ai.Diagnosis_Tag.RID,
    eye_ai.Diagnosis_Tag.Name,
    eye_ai.Diagnosis_Tag.Description))


,RID,Name,Description
0,2-1S0R,UI Annotation,UI Annotation
1,C1T4,Initial Diagnosis,The diagnosis is extracted from the original d...
2,2-35G0,GlaucomaSuspect,Glaucoma suspect study
3,2-35RM,AI_glaucomasuspect_test,Diagnosis tag for TEST dataset for AI algorith...
4,2-4F74,GlaucomaSuspect-Training,Glaucoma suspect study on [Graded Training dat...
5,2-4F76,GlaucomaSuspect-Validation,Glaucoma suspect study on [Graded Validation d...


In [25]:
# @title Configure
diagnosis = eye_ai.Diagnosis  # All of the diagnosis in the catalog
dataset_rid = "2-277M" # @param {type:"string"}
diagnosis_tag_rid = "2-35RM" # @param {type:"string"}
compare_value = "Image_Quality" #@param {type:"string"}["Diagnosis", "Image_Quality", "Cup/Disk_Ratio"]
initial_diagnosis_tag_rid = "C1T4"
angle_two_rid = '2SK6'

dataset = eye_ai.Dataset
ds = dataset.path.filter(dataset.RID==dataset_rid).attributes(dataset.RID, dataset.Description).fetch()[0]
print(f"Dataset tag (RID:{dataset_rid}):\n\tdescription: {ds['Description']}")

diagnosis_tag = eye_ai.Diagnosis_Tag
diagnosis_info = diagnosis_tag.path.filter(diagnosis_tag.RID==diagnosis_tag_rid).attributes(diagnosis_tag.RID, diagnosis_tag.Name, diagnosis_tag.Description).fetch()[0]
print(f"Diagnosis tag (RID:{diagnosis_tag_rid}):\n\tname {diagnosis_info['Name']} \n\tdescription: {diagnosis_info['Description']}")

print(f"In the wide table, {compare_value} will be compared among graders and the initial diagnosis.")

Dataset tag (RID:2-277M):
	description: **Graded Test Dataset** A random subset of subjects from the [test](https://www.eye-ai.org/id/eye-ai/2-277C) dataset. Contains 250 subjects from each diagnosis category.
Diagnosis tag (RID:2-35RM):
	name AI_glaucomasuspect_test 
	description: Diagnosis tag for TEST dataset for AI algorithm to identify glaucoma suspects in LAC DHS dataset (1000 images from 250 glaucoma suspects and 250 normals)
In the wide table, Image_Quality will be compared among graders and the initial diagnosis.


In [26]:
# @title Example of Graded Test Dataset

Graded_test_initial_diag = EA.image_tall(dataset_rid, initial_diagnosis_tag_rid)
Graded_test_grader_diag = EA.image_tall(dataset_rid, diagnosis_tag_rid)
# Graded_test_gold = EA.image_tall(dataset_rid, "2-5GXP")
long, wide = Reshape_table([Graded_test_initial_diag, Graded_test_grader_diag], compare_value)


long.to_csv("GradedTest_diag_long.csv")
wide.to_csv("GradedTest_diag_wide.csv")
# Graded_test_subject_image_quality.to_csv("GradedTest_sub_image_quality.csv")

In [28]:
long

,index,Subject_RID,Diagnosis_RID,Full_Name,Image,Image_Side,Diagnosis,Cup/Disk_Ratio,Image_Quality
0,0,70GG,HBHW,Initial Diagnosis,7P0A,Left,No Glaucoma,NaN,Good
1,1,70GG,HBHY,Initial Diagnosis,7P0C,Right,No Glaucoma,NaN,Good
2,2,6YWM,HBXM,Initial Diagnosis,7P9A,Left,No Glaucoma,NaN,Bad
3,3,6YWM,HBXW,Initial Diagnosis,7P9J,Right,No Glaucoma,NaN,Good
4,4,6VP8,HC3J,Initial Diagnosis,7PE8,Left,No Glaucoma,NaN,Good
...,...,...,...,...,...,...,...,...,...
12995,11995,73HT,2-4HB0,Van Nguyen,878E,Left,No Glaucoma,0.4,Good
12996,11996,718P,2-4HB2,Van Nguyen,8HH8,Right,Suspected Glaucoma,0.6,Bad
12997,11997,7052,2-4HB4,Van Nguyen,8RPE,Left,No Glaucoma,0.2,Good
12998,11998,6X6M,2-4HB6,Van Nguyen,9A6A,Right,No Glaucoma,0.5,Good


In [29]:
wide

,,Full_Name,Aidan Lee,Alanna James,Andrew Duong,Austin Bohner,Benjamin Xu,Brandon Wong,Haroon Rasheed,Initial Diagnosis,Jeffrey Gluckstein,Kendra Hong,Kent Nguyen,Rahul Dhodapkar,Van Nguyen
Image,Image_Side,Subject_RID,,,,,,,,,,,,,
7P0A,Left,70GG,Good,Bad,Good,Good,Good,Good,Good,Good,Good,Good,Good,Good,Good
7P0C,Right,70GG,Good,Bad,Good,Good,Good,Good,Good,Good,Bad,Good,Good,Good,Good
7P9A,Left,6YWM,Good,Bad,Good,Good,Good,Bad,Bad,Bad,Bad,Bad,Bad,Good,Good
7P9J,Right,6YWM,Good,Good,Good,Good,Good,Good,Good,Good,Good,Good,Good,Good,Good
7PE8,Left,6VP8,Bad,Bad,Bad,Bad,Bad,Bad,Bad,Good,Bad,Bad,Bad,Bad,Bad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AH5P,Left,72A6,Good,Good,Good,Good,Good,Good,Good,Good,Good,Good,Bad,Good,Good
AH6P,Right,74R4,Bad,Bad,Good,Bad,Bad,Good,Good,Good,Bad,Good,Bad,Bad,Good
AH6R,Left,74R4,Bad,Bad,Good,Bad,Bad,Good,Good,Good,Bad,Good,Bad,Good,Bad
